# Diffusion Model Project

This notebook contains a basic animation of the denoising diffusion process and a comparison of different timestep values.

What you can change in the `src/config.yaml`:
- You can select the model (and dataset) used by editing the `model` attribute
- You can select the number of generation time steps by editing the `model.timesteps` attribute

In [ ]:
import functions
import diffusers
import torch
from tqdm import tqdm

In [ ]:
# setup
model_id = functions.config("model")
model = diffusers.UNet2DModel.from_pretrained(model_id)
ddpm_scheduler = diffusers.DDPMScheduler.from_pretrained(model_id)
ddpm_scheduler.set_timesteps(functions.config("model.timesteps"))

## Denoising Animation

This part shows a basic animation of the denoising diffusion process.

If the generation takes too long, try to decrease the `model.timesteps` attribute in the `src/config.yaml` file. If the generated image look too bad, try to increase the `model.timesteps` attribute.

In [ ]:
# input prepraration
image_size = model.config.sample_size # get image size
noise = functions.generate_noise(image_size) # sample random noise

In [ ]:
# output generation
current = noise
history = [noise]
for i, t in enumerate(ddpm_scheduler.timesteps):
    with torch.no_grad():
        predicted_noise = model(current, t).sample
        current = ddpm_scheduler.step(predicted_noise, t, current).prev_sample
        labels = [f"Image {i + 1}/{functions.config('model.timesteps')}", "Predicted Noise", "Image - Predicted Noise"]
        functions.show_images(history[-1], predicted_noise, current, labels=labels)
        history.append(current)

In [ ]:
# show images
functions.show_images(*history[::functions.config('model.timesteps')//5])

If you want to save the generated image, you can execute the following cell. Consider choosing a reasonable file name to avoid overwriting potentially existing files.

In [ ]:
# save image
functions.tensor_as_image(current).save("../output/output.png")

## DDIM Accelerated Sampling

This part compares different numbers of diffusion timesteps. Feel free to adjust the number of timesteps to compare ot to add more values to the list.

In [ ]:
# setup
ddim_scheduler = diffusers.DDIMScheduler.from_pretrained(model_id)
timesteps = [10, 50, 1000]

In [ ]:
# output generation
images = [noise for _ in range(len(timesteps))]
for i in range(len(timesteps)):
    ddim_scheduler.set_timesteps(timesteps[i])
    for t in tqdm(ddim_scheduler.timesteps):
        with torch.no_grad():
            predicted_noise = model(images[i], t).sample
            images[i] = ddim_scheduler.step(predicted_noise, t, images[i]).prev_sample
            functions.show_images(*images)